In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
import sys
import os
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from scipy import stats
from utils.value_change_utils import open_experiment, CustomAlignedDataRewardBlocks, get_all_experimental_records, get_block_change_info, add_traces_and_peaks, plot_mean_trace_for_condition_value_switch, one_session_get_block_changes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
root_directory = 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis\\utils\\'
processed_data_dir = os.path.join(root_directory, 'value_change_data')
block_data_file = os.path.join(processed_data_dir, 'value_switch_nacc_mice.csv')
all_reward_block_data = pd.read_pickle(block_data_file)

In [3]:
sorted_data = all_reward_block_data.sort_values(['mouse', 'session', 'trial number']).reset_index(drop=True)
sorted_data['block switches'] = sorted_data['block number'] - sorted_data['block number'].shift()
sorted_data['new sessions'] = sorted_data['session'].ne(sorted_data['session'].shift().bfill()).astype(int)
sorted_data.iloc[0, sorted_data.columns.get_loc('new sessions')] = 1
sorted_data.loc[sorted_data['new sessions'] == 1, 'block switches'] = 1
block_switch_inds = sorted_data.loc[sorted_data['block switches'] != 0].reset_index(drop=True)

In [4]:
num_trials_to_look_at = 30
min_num_trials = 50
traces = []
peaks = []
trial_nums = []
block_inds = []
rel_reward_amounts = []
reward_amounts = []
mouse_ids = []
session_ids = []
num_blocks = block_switch_inds.index.values.shape[0]
for block_num, block in block_switch_inds.iterrows():
    first_trial = block['trial number']
    mouse = block['mouse']
    session = block['session']
    block_id = block['block number']
    block_switch_trial_num = block['trial number']
    all_session_trials = sorted_data[(sorted_data['mouse'] == mouse) & (sorted_data['session'] == session)]
    all_block_trials = all_session_trials[all_session_trials['block number'] == block_id]
    if block_id == 0 or all_block_trials.shape[0]>=min_num_trials:
        last_trials_of_block = all_block_trials[-num_trials_to_look_at:]
        mean_trace = last_trials_of_block['traces'].apply(np.mean)
        avg_trace = last_trials_of_block.groupby(['mouse', 'contra reward amount'])['traces'].apply(np.mean).values[0]
        decimated = decimate(avg_trace[int(len(avg_trace)/2):], 10) 
        peak_idx = peakutils.indexes(decimated)[0] 
        peak = decimated[peak_idx]
        traces.append(decimated)
        peaks.append(peak)
        trial_nums.append(last_trials_of_block['trial number'].values)
        rel_reward_amounts.append(last_trials_of_block['relative reward amount'].values[0])
        reward_amounts.append(last_trials_of_block['contra reward amount'].values[0])
        block_inds.append(last_trials_of_block.index.values[0])
        mouse_ids.append(mouse)
        session_ids.append(session)

In [273]:
num_trials_to_look_at = 30
min_num_trials = 30
traces = []
peaks = []
trial_nums = []
block_inds = []
rel_reward_amounts = []
reward_amounts = []
mouse_ids = []
session_ids = []
num_blocks = block_switch_inds.index.values.shape[0]
for block_num, block in block_switch_inds.iterrows():
    first_trial = block['trial number']
    mouse = block['mouse']
    session = block['session']
    block_id = block['block number']
    block_switch_trial_num = block['trial number']
    all_session_trials = sorted_data[(sorted_data['mouse'] == mouse) & (sorted_data['session'] == session)]
    all_block_trials = all_session_trials[all_session_trials['block number'] == block_id]

    if block_id == 0:
        last_trials_of_block = all_block_trials[all_block_trials['trial number'] >= block_switch_trial_num + 10]
    else:
        last_trials_of_block = all_block_trials[all_block_trials['trial number'] >= block_switch_trial_num + 100]
        
    if last_trials_of_block.shape[0] > min_num_trials:
        mean_trace = last_trials_of_block['traces'].apply(np.mean)
        avg_trace = last_trials_of_block.groupby(['mouse', 'contra reward amount'])['traces'].apply(np.mean).values[0]
        decimated = decimate(avg_trace[int(len(avg_trace)/2):], 10) 
        peak_idx = peakutils.indexes(decimated)[0] 
        peak = decimated[peak_idx]
        traces.append(decimated)
        peaks.append(peak)
        trial_nums.append(last_trials_of_block['trial number'].values)
        rel_reward_amounts.append(last_trials_of_block['relative reward amount'].values[0])
        reward_amounts.append(last_trials_of_block['contra reward amount'].values[0])
        block_inds.append(last_trials_of_block.index.values[0])
        mouse_ids.append(mouse)
        session_ids.append(session)

In [5]:
avg_block_data = {}
avg_block_data['block id'] = block_inds
avg_block_data['peaks'] = peaks
avg_block_data['relative reward amount'] = rel_reward_amounts
avg_block_data['contra reward amount'] = reward_amounts
avg_block_data['mouse'] = mouse_ids
avg_block_data['session'] = session_ids
avg_block_dataf = pd.DataFrame(avg_block_data)

In [6]:
avg_block_dataf['avg traces'] = pd.Series(traces, index=avg_block_dataf.index)
df_for_plot = avg_block_dataf.groupby(['mouse', 'session', 'relative reward amount', 'contra reward amount'])['peaks'].apply(np.mean)
df_for_plot = df_for_plot.reset_index()

In [7]:
mice = []
reward_changes = []
cue_changes = []
session_ids = []

first_blocks = df_for_plot[df_for_plot['relative reward amount'] == 0]
for first_block_ind, first_block in first_blocks.iterrows():
    mouse = first_block['mouse']
    session = first_block['session']
    session_blocks = df_for_plot[np.logical_and(df_for_plot['mouse'] == mouse, df_for_plot['session'] == session)]
    other_block = session_blocks.loc[session_blocks['relative reward amount'] != 0]
    if other_block.shape[0] > 0:
        mice.append(mouse)
        reward_changes.append(other_block['relative reward amount'].values[0])
        cue_change = (other_block['peaks'] - first_block['peaks']).values[0]
        cue_changes.append(cue_change)
        session_ids.append(session)
diff_block_data = {}
diff_block_data['mouse'] = mice
diff_block_data['reward size change'] = reward_changes
diff_block_data['cue size change'] = cue_changes
diff_block_data['session'] = session_ids
diff_block_dataf = pd.DataFrame(diff_block_data)

In [8]:
diff_block_dataf

,mouse,reward size change,cue size change,session
0,SNL_photo31,-4.0,-0.373250,20210126
1,SNL_photo31,-4.0,-1.049855,20210127
2,SNL_photo31,4.0,0.507705,20210128
3,SNL_photo32,4.0,-0.219052,20210126
4,SNL_photo32,-4.0,-0.604870,20210127
5,SNL_photo32,-4.0,-0.160719,20210128
6,SNL_photo34,4.0,0.302136,20210126
7,SNL_photo34,4.0,0.740962,20210127
8,SNL_photo34,4.0,0.587651,20210128
9,SNL_photo35,4.0,0.347043,20210126


In [9]:
df_for_plot1 = diff_block_dataf.groupby(['mouse', 'reward size change'])['cue size change'].apply(np.mean)
df_for_plot1 = df_for_plot1.reset_index()

In [10]:
df_for_plot1

,mouse,reward size change,cue size change
0,SNL_photo31,-4.0,-0.711553
1,SNL_photo31,4.0,0.507705
2,SNL_photo32,-4.0,-0.382795
3,SNL_photo32,4.0,-0.219052
4,SNL_photo34,4.0,0.543583
5,SNL_photo35,-4.0,-0.426362
6,SNL_photo35,4.0,0.350593


In [73]:
df1 = df_for_plot1.pivot(index='reward size change', columns='mouse', values='cue size change').sort_values('reward size change', ascending=False)

In [74]:
df1

mouse,SNL_photo31,SNL_photo32,SNL_photo34,SNL_photo35
reward size change,,,,
4.0,0.507705,-0.219052,0.543583,0.350593
-4.0,-0.711553,-0.382795,NaN,-0.426362


In [35]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    
    fg = sns.FacetGrid(data=data, hue='mouse', aspect=1.61)
    fg.map(plt.scatter, 'reward size change', 'cue size change', s=6)



In [71]:
fg = sns.FacetGrid(data=df_for_plot1, hue='mouse', aspect=0.8, palette='viridis')
fg.map(plt.scatter, 'reward size change', 'cue size change', s=10)
ax=fg.axes[0][0]
ax.axhline(0, color='k', lw=0.5)
ax.set_xticks([-4, 4])
ax.set_xlabel('relative value')
plt.ylabel('change in fluorescence z-score')
plt.tight_layout()

In [52]:
fg.axes

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000000B0B9C1D0>]],
      dtype=object)